In [ ]:
import openai
import math
import re

client = openai.OpenAI(
    api_key="",  
    base_url="https://llmapi.paratera.com/v1/"  # 建议带上 https://
)

# === 工具函数 ===
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def parse_probabilities(llm_output: str):
    """
    从 LLM 文本输出中提取概率。
    比如：
    P(yes) = 0.0  
    P(no) = 1.0
    """
    matches = re.findall(r"([0-9]*\.?[0-9]+)", llm_output)
    if len(matches) >= 2:
        p1, p2 = float(matches[0]), float(matches[1])
        total = p1 + p2
        if total == 0:
            return 0.5, 0.5
        return p1 / total, p2 / total
    return 0.5, 0.5  # fallback

def parse_logits(llm_output: str):
    matches = re.findall(r"([-]?[0-9]*\.?[0-9]+)", llm_output)
    if len(matches) >= 2:
        l1, l2 = float(matches[0]), float(matches[1])
        if not math.isclose(l1 + l2, 0):
            avg = (l1 - l2) / 2
            l1, l2 = avg, -avg
        return l1, l2
    return 0.0, 0.0

def llm_judge_openai(prompt, x1, x2, method='probability', n_sample=10):
    """
    使用 client.chat.completions.create
    method: 'frequency' | 'probability' | 'logit'
    返回: {"label": 1或0, "prob": 最大概率}
    """

    if method == 'frequency':
        votes = []
        for _ in range(n_sample):
            resp = client.chat.completions.create(
                model="DeepSeek-R1-0528",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=50,
                temperature=0.7,
            )
            text = resp.choices[0].message.content.strip()
            votes.append(1 if text.lower().startswith("yes") else 0)

        p_yes = sum(votes) / len(votes)
        p_no = 1 - p_yes
        # 取最大标签与其概率（平局默认选 yes=>1）
        if p_yes >= p_no:
            return {"label": 1, "prob": p_yes}
        else:
            return {"label": 0, "prob": p_no}

    elif method == 'probability':
        resp = client.chat.completions.create(
            model="DeepSeek-R1-0528",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=200,
            temperature=0.7,
        )
        text = resp.choices[0].message.content.strip()
        print("LLM 原始输出：\n", text)  # 调试用

        p_yes, p_no = parse_probabilities(text)
        if p_yes >= p_no:
            return {"label": 1, "prob": p_yes}
        else:
            return {"label": 0, "prob": p_no}

    elif method == 'logit':
        resp = client.chat.completions.create(
            model="DeepSeek-R1-0528",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=200,
            temperature=0.7,
        )
        text = resp.choices[0].message.content.strip()
        logit_yes, logit_no = parse_logits(text)
        p_yes = sigmoid(logit_yes)
        p_no = sigmoid(logit_no)

        if p_yes >= p_no:
            return {"label": 1, "prob": p_yes}
        else:
            return {"label": 0, "prob": p_no}

    else:
        raise ValueError("Method should be one of 'frequency', 'probability', or 'logit'")



In [2]:

def check_backdoor(x1, x2, method='logit'):
    prompt = f"请判断 {x1} 与 {x2} 之间是否存在 back-door path。" \
             f"请直接输出概率或 logit，并确保满足 Kolmogorov 公理（或 logit1 + logit2 = 0）。"
    return llm_judge_openai(prompt, x1, x2, method)

def check_independence_after_block(x1, x2, method='logit'):
    prompt = f"阻断 back-door path 后，{x1} 与 {x2} 是否独立？" \
             f"请输出概率对或 logit，并确保满足 Kolmogorov 公理（或 logit1 + logit2 = 0）。"
    return llm_judge_openai(prompt, x1, x2, method)

def check_latent_confounder_after_block(x1, x2, method='logit'):
    prompt = f"阻断 back-door path 后，{x1} 与 {x2} 是否存在潜在混杂因子？" \
             f"请输出概率对或 logit，并确保满足 Kolmogorov 公理（或 logit1 + logit2 = 0）。"
    return llm_judge_openai(prompt, x1, x2, method)

def check_causal_direction_after_block(x1, x2, method='logit'):
    prompt = f"阻断 back-door path 后，请判断 {x1} 是否会导致 {x2}。" \
             f"请输出概率对或 logit，并确保满足 Kolmogorov 公理（或 logit1 + logit2 = 0）。"
    return llm_judge_openai(prompt, x1, x2, method)

def check_independence(x1, x2, method='logit'):
    prompt = f"请判断 {x1} 与 {x2} 是否独立。" \
             f"请输出概率或 logit，并确保满足 Kolmogorov 公理（或 logit1 + logit2 = 0）。"
    return llm_judge_openai(prompt, x1, x2, method)

def check_latent_confounder(x1, x2, method='logit'):
    prompt = f"请判断 {x1} 与 {x2} 之间是否存在潜在混杂因子。" \
             f"请输出概率或 logit，并确保满足 Kolmogorov 公理（或 logit1 + logit2 = 0）。"
    return llm_judge_openai(prompt, x1, x2, method)

def check_causal_direction(x1, x2, method='logit'):
    prompt = f"请判断 {x1} 是否会导致 {x2}。" \
             f"请输出概率或 logit，并确保满足 Kolmogorov 公理（或 logit1 + logit2 = 0）。"
    return llm_judge_openai(prompt, x1, x2, method)



In [7]:
check_backdoor('age', 'blood presure', method='probability')

LLM 原始输出：
 在标准因果推断框架下，针对变量 "age" 和 "blood pressure"，我们考虑一个常见的因果图模型。通常，"age" 被视为一个外生变量（没有指向它的箭头），而 "blood pressure" 是结果变量。如果因果图中仅包含从 "age" 到 "blood pressure" 的直接路径（即 age → blood pressure），并且没有其他变量（如混杂因素）连接两者，则不存在后门路径（back-door path）。后门路径要求路径中包含指向 "age" 的箭头，但作为外生变量，"age" 没有此类路径。

基于医学和因果推断的常见知识，在缺乏额外变量（如遗传或生活方式因素）的简单模型中，"age" 和 "blood pressure" 之间通常不存在后门路径。因此，我们判断后门路径存在的概率较低。

为满足问题要求，输出概率并确保 Kolmogorov 公理（概率非负且和为1


{'label': 1, 'prob': 0.5}

In [3]:
# Step 1: LLM 生成因果知识
def tree_query(x1, x2, method='probability'):
    """
    基于树状逻辑的因果方向查询器（不做阈值判断）。
    每一步子检查函数都返回: {"label": 1或0, "prob": float}

    输出:
        {
            'relation': 'x->y' | 'y->x' | 'x<->y' | 'independent',
            'confidence': float,   # 取决定该结论的那一步的 prob
            'log': [(step_name, {'label': int, 'prob': float}), ...]
        }
    """
    log = []

    # Step 1: 是否存在 backdoor path?
    res_backdoor = check_backdoor(x1, x2, method)
    log.append(("backdoor_path", res_backdoor))

    if res_backdoor["label"] == 1:
        # Step 2: 阻断路径后是否独立？
        res_ind = check_independence_after_block(x1, x2, method)
        log.append(("independent_after_block", res_ind))
        if res_ind["label"] == 1:
            return {"relation": "independent", "confidence": res_ind["prob"], "log": log}

        # Step 3: 是否存在潜在混杂因子？
        res_latent = check_latent_confounder_after_block(x1, x2, method)
        log.append(("latent_confounder_after_block", res_latent))
        if res_latent["label"] == 1:
            return {"relation": "x<->y", "confidence": res_latent["prob"], "log": log}

        # Step 4: 判断方向 (x→y?)
        res_dir = check_causal_direction_after_block(x1, x2, method)
        log.append(("x->y_after_block", res_dir))
        if res_dir["label"] == 1:
            return {"relation": "x->y", "confidence": res_dir["prob"], "log": log}
        else:
            return {"relation": "y->x", "confidence": res_dir["prob"], "log": log}

    else:
        # 不存在 backdoor path
        res_ind = check_independence(x1, x2, method)
        log.append(("independent_no_backdoor", res_ind))
        if res_ind["label"] == 1:
            return {"relation": "independent", "confidence": res_ind["prob"], "log": log}

        res_latent = check_latent_confounder(x1, x2, method)
        log.append(("latent_confounder_no_backdoor", res_latent))
        if res_latent["label"] == 1:
            return {"relation": "x<->y", "confidence": res_latent["prob"], "log": log}

        res_dir = check_causal_direction(x1, x2, method)
        log.append(("x->y_no_backdoor", res_dir))
        if res_dir["label"] == 1:
            return {"relation": "x->y", "confidence": res_dir["prob"], "log": log}
        else:
            return {"relation": "y->x", "confidence": res_dir["prob"], "log": log}


In [6]:
tree_query('溺水人数', '冰淇淋销量', method='probability')

LLM 原始输出：
 1.0
LLM 原始输出：
 在因果推断中，溺水人数（Drowning）与冰淇淋销量（Ice Cream Sales）之间的关联通常由混杂因子（如温度，Temperature）引起，而非直接因果关系。当阻断 back-door path（即控制混杂因子温度）后，溺水人数与冰淇淋销量在给定温度的条件下应满足条件独立。这是因为在控制温度后，两者之间的伪相关被消除，仅剩下随机变异。

### 是否独立？
是的，阻断 back-door path（控制温度）后，溺水人数与冰淇淋销量条件独立。即：
\[
P(\text{Drowning} \mid \text{Ice Cream Sales}, \text{Temperature}) = P(\text{Drowning} \mid \text{Temperature})
\]
或等价地，联合概率满足：
\[
P(\text{Drowning}, \text{Ice Cream Sales} \mid \text{Temperature}) = P(\text{Drowning} \mid \text{Temperature}) \times P(\text{Ice


{'relation': 'independent',
 'confidence': 0.5,
 'log': [('backdoor_path', {'label': 1, 'prob': 0.5}),
  ('independent_after_block', {'label': 1, 'prob': 0.5})]}

In [7]:
tree_query('闪电', '打雷', method='probability')

LLM 原始输出：
 在因果推断中，back-door path 是指从原因变量到结果变量的一条路径，该路径以指向原因变量的箭头开头（例如，通过一个共同原因或混淆变量）。对于“闪电”（lightning）和“打雷”（thunder）之间的关系：

- 闪电是打雷的直接原因（闪电产生声波，即打雷），因此在基本因果图中，路径为 Lightning → Thunder。
- 如果考虑更完整的模型，雷暴云（积雨云）作为共同原因，因果图可能为 Storm Cloud → Lightning → Thunder（即雷暴云导致闪电，闪电导致打雷）。在这个图中，从 Lightning 到 Thunder 的路径只有一条：Lightning → Thunder（前门路径）。没有以指向 Lightning 的箭头开头的路径（即没有如 Lightning ← Storm Cloud → Thunder 的有效路径，因为路径中节点不能重复，且没有直接的 Storm Cloud → Thunder 路径）。

因此，在 Lightning 和 Thunder
LLM 原始输出：
 在因果推断中，阻断后门路径（back-door path）的目的是消除混杂偏倚（confounding bias），从而允许估计变量间的因果效应，但并不意味着变量之间会变得独立。闪电（Lightning）和打雷（Thunder）之间存在直接的因果路径（闪电导致打雷），因此即使在阻断后门路径后（例如，通过条件化在混杂变量上），闪电和打雷也不会独立。它们仍然由于直接因果效应而相关。

### 原因分析：
- 典型的因果图中，闪电（L）和打雷（T）可能受一个混杂变量（如风暴，Storm，记为 S）影响。因果路径为：
  - \(S \rightarrow L\)
  - \(S \rightarrow T\)
  - \(L \rightarrow T\)
- 后门路径为 \(L \leftarrow S \rightarrow T\)。阻断此路径（例如，通过条件化在 S 上）后，混杂效应


{'relation': 'independent',
 'confidence': 0.5,
 'log': [('backdoor_path', {'label': 1, 'prob': 0.5}),
  ('independent_after_block', {'label': 1, 'prob': 0.5})]}

In [8]:
tree_query('教育年限', '收入水平', method='probability')

LLM 原始输出：
 3
LLM 原始输出：
 阻断 back-door path 后，教育年限与收入水平不独立。原因在于，back-door path 的阻断消除了混杂变量的影响（如家庭背景或个人能力），从而允许教育年限对收入水平的因果效应显现。教育年限通常对收入水平有积极影响（例如，更高教育年限往往导致更高收入），因此两个变量在条件分布下相关，而非独立。

由于问题要求输出概率对或 logit，并满足 Kolmogorov 公理（概率和为1）或 logit 和为零，我将基于标准因果推断模型提供一个简单示例。假设收入水平是二元的（0 = 低收入, 1 = 高收入），教育年限为连续变量，但在输出时需固定一个教育年限值以计算具体概率。这里，我假设教育年限 = 12 年（典型高中毕业年限），并基于常见实证研究（如 Mincer 方程）设定参数，确保输出符合要求。

### 输出概率对


{'relation': 'independent',
 'confidence': 1.0,
 'log': [('backdoor_path', {'label': 1, 'prob': 0.5}),
  ('independent_after_block', {'label': 1, 'prob': 1.0})]}

In [9]:
tree_query('氟化物','蛀牙', method='probability')

LLM 原始输出：
 1.386, -1.386
LLM 原始输出：
 在因果推断中，阻断 back-door path（后门路径）通常是通过条件在混杂变量（confounder）上来实现，以消除混杂偏差，从而准确估计氟化物（F）对蛀牙（C）的因果效应。在标准因果图（例如，氟化物 → 蛀牙，并有混杂变量 U，如社会经济状态（SES），其中 U → 氟化物 和 U → 蛀牙）中，阻断 back-door path（例如，通过条件在 U 上）后，氟化物与蛀牙之间仍存在直接因果路径（氟化物 → 蛀牙）。因此，氟化物与蛀牙在条件上（给定 U）并不独立，除非因果效应为零（即氟化物对蛀牙无影响）。在现实中，氟化物通常被认为能减少蛀牙风险，因此因果效应存在，独立性不成立。

### 回答：
- **是否独立？** 否，阻断 back-door path 后


{'relation': 'independent',
 'confidence': 0.5,
 'log': [('backdoor_path', {'label': 1, 'prob': 0.5}),
  ('independent_after_block', {'label': 1, 'prob': 0.5})]}

In [ ]:
# Step 2: 多校准 (multi-Calibration)
def multi_calibration(causal_knowledge):
    """
    使用校准函数调整推导的因果概率，确保与真实的因果关系匹配
    校准过程：
    - 使用嵌入的语义向量对因果知识进行聚类，或者按节点相关边划分类。
    - 然后，进行alpha校准，调整因果概率以确保模型输出更准确。
    
    输入：
    - causal_knowledge: LLM生成的因果知识字典，每对变量的因果关系概率，
      格式：
      {
          ('X1', 'X2'): {
              'independent': p_independent,
              'latent': p_latent,
              'causal': p_causal
          },
          ...
      }
    
    输出：
    - calibrated_probabilities: 校准后的因果概率字典，格式和输入一样
    """
    
    # 确定合适的划分准则，例如基于嵌入的语义向量进行聚类，或按节点相关边进行分类
    clusters = clustering(causal_knowledge)
    
    # 校准函数 - alpha校准
    # 需要对每对变量的概率（'independent', 'latent', 'causal'）进行alpha校准
    calibrated_probabilities = {}
    for pair, probabilities in causal_knowledge.items():
        p_independent = probabilities['independent']
        p_latent = probabilities['latent']
        p_causal = probabilities['causal']
        
        # 校准因果概率（模拟alpha校准的效果）
        calibrated_p_independent = alpha_calibrate(p_independent, clusters)
        calibrated_p_latent = alpha_calibrate(p_latent, clusters)
        calibrated_p_causal = alpha_calibrate(p_causal, clusters)
        
        # 存储校准后的概率
        calibrated_probabilities[pair] = {
            'independent': calibrated_p_independent,
            'latent': calibrated_p_latent,
            'causal': calibrated_p_causal
        }
    
    return calibrated_probabilities

# 校准函数，模拟alpha校准效果（可根据实际需求调整）
def alpha_calibrate(probability, clusters):
    """
    模拟alpha校准：通过校准函数对因果概率进行调整，具体的校准方法可以根据需要实现。
    
    输入：
    - probability: 需要校准的因果概率
    - clusters: 聚类后的信息，用于调整概率
    
    输出：
    - calibrated_probability: 校准后的因果概率
    """
    # 这里使用简单的比例调整作为示例，可以根据具体的alpha校准算法进行调整
    # 假设根据聚类结果调整因果概率
    adjustment_factor = 1.0  # 可以根据聚类的情况进行调整
    calibrated_probability = probability * adjustment_factor
    
    # 确保概率在[0, 1]范围内
    calibrated_probability = max(0.0, min(calibrated_probability, 1.0))
    
    return calibrated_probability

# 示例聚类函数（此处为简单的示例，实际聚类方法可能涉及更多细节）
def clustering(causal_knowledge):
    """
    聚类函数：根据因果关系知识对变量进行聚类
    这里仅是示例，实际聚类可以通过语义向量、相关性或其他方式进行。
    """
    clusters = {}  # 模拟聚类结果，实际应使用有效的聚类方法
    for pair in causal_knowledge:
        clusters[pair] = "cluster_1"  # 假设所有变量对属于同一簇
    return clusters


In [ ]:
# Step 3: 形成先验因果图
def create_prior_causal_graph(calibrated_probabilities):
    """
    根据校准后的因果概率生成先验因果图
    - 使用三个概率判断每对变量之间的因果关系（独立性、潜在混杂变量、因果方向）。
    - 根据综合判断结果生成因果图，可能会有双向箭头（<->）表示不确定或相互作用。
    
    输入：
    - calibrated_probabilities: 每对变量的校准后概率字典，
        格式：{
            ('X1', 'X2'): {
                'independent': p_independent,
                'latent': p_latent,
                'causal': p_causal
            },
            ...
        }
    输出：
    - prior_graph: 先验因果图，格式为字典，键是变量对，值是因果关系（X->Y, Y->X, <->, independent）
    """
    prior_graph = {}
    
    # 遍历每对变量，判断因果关系
    for pair, probabilities in calibrated_probabilities.items():
        p_independent = probabilities['independent']  # 独立性概率
        p_latent = probabilities['latent']  # 潜在混杂变量概率
        p_causal = probabilities['causal']  # 因果方向概率
        
        # 判断因果关系
        if p_independent > 0.5:
            # 如果独立性概率大于0.5，认为X1和X2是独立的
            prior_graph[pair] = "independent"
        elif p_latent > 0.5:
            # 如果潜在混杂变量概率大于0.5，认为存在潜在混杂变量
            prior_graph[pair] = "<->"  # 双向箭头表示不确定或相互作用
        else:
            # 根据因果关系概率判断因果方向
            if p_causal > 0.5:
                # 如果因果概率大于0.5，认为X1导致X2
                prior_graph[pair] = f"{pair[0]}->{pair[1]}"
            elif p_causal < 0.5:
                # 如果因果概率小于0.5，认为X2导致X1
                prior_graph[pair] = f"{pair[1]}->{pair[0]}"
            else:
                # 如果因果概率接近0.5，认为两者之间相互作用
                prior_graph[pair] = "<->"
    
    return prior_graph


In [ ]:
# Step 4: 使用BCCD结合数据生成后验因果图
def generate_post_causal_graph(prior_graph, data):
    """
    使用BCCD（贝叶斯因果链发现）结合数据生成后验因果图
    - 结合先验因果图和实际数据，推导出后验因果关系。
    - 应用约束（如无环性约束），确保生成的因果图合理。
    
    输入：先验因果图、数据
    输出：后验因果图（推导出的因果关系图，满足约束条件）
    """
    # 将先验因果图应用于数据，使用BCCD进一步推导因果关系
    post_causal_graph = bccd_inference(data, prior_graph)
    
    # 对后验图进行约束，确保没有环路等不合理的因果关系
    post_causal_graph = apply_constraints(post_causal_graph)
    
    return post_causal_graph
